<a href="https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Study 1: Designing explainable speech-based machine learning models of depression

Given that Colab doesn't automatically load any of the content (data or other functions) from the Github repo, running the code below will copy the repo into the workspace directory for use. To save this ipynb file back to Github, select **File > Save** (which should show the repo if you're signed in) or **File > Save a copy in Github** if it's in the menu.

Note that the content of the data files or any of the other file structures are not saved back to Github, so make sure that if you make changes to things there, that you put them in Github separately.

In [ ]:
# Clone Github Repo into the temporary local environment so data can be accessed and manipulated
!git clone https://github.com/cgenevier/CSCI5622-HW4.git
%cd CSCI5622-HW4

Cloning into 'CSCI5622-HW4'...
remote: Enumerating objects: 391, done.
remote: Counting objects: 100% (391/391), done.
remote: Compressing objects: 100% (389/389), done.
remote: Total 391 (delta 0), reused 391 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (391/391), 5.13 MiB | 21.73 MiB/s, done.
/content/CSCI5622-HW4


In [ ]:
# Import necessary libraries

# Pandas, seaborn, and numpy for data manipulation
import pandas as pd
pd.set_option("display.max_rows", None)
import statistics as stat
import seaborn as sns
import numpy as np
np.random.seed(42)

# Keras & TensorFlow for building the neural networks
import itertools, json, time
from itertools import count
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, callbacks, backend as K
tf.random.set_seed(42)

# Matplotlib for graphing
import matplotlib.pyplot as plt

In [ ]:
# Confirm we can load a dataset (can delete this later)
depression_labels = pd.read_csv("data/DepressionLabels.csv")
example_accoustic = pd.read_csv("data/E-DAIC_Acoustics/386_utterance_agg.csv")

print(depression_labels.head())
print(example_accoustic.head())

   Participant_ID  PHQ_Score
0             300          2
1             301          3
2             302          4
3             303          0
4             304          6
   ParticipantID  UtteranceIndex  Start_Time  End_Time  \
0            386               1         8.3      10.4   
1            386               2        20.3      26.3   
2            386               3        46.1      46.9   
3            386               4        51.3      53.3   
4            386               5        58.5      64.6   

                                                Text  Confidence  \
0                   might have pulled something that    0.900792   
1        I'm going to bring the great thanks so much    0.964655   
2                                         and please    0.830017   
3                         are you okay with this yes    0.987629   
4   oh I'm fine I'm a little tired but I found ou...    0.907408   

   Loudness_sma3  alphaRatio_sma3  hammarbergIndex_sma3  slope0-500_

### (a) (2 points) Extracting language features.

### (b) (2 points) Estimating depression severity with interpretable models using language features.

### (c) (2 points) Estimating depression severity with interpretable models using acoustic features.

### (d) (2 points) Estimating depression severity with unimodal and multimodal deep learning models.

### (e) (2 points) Explainable ML.

### (f) (Bonus, 2 points) Experimenting with transformers.